# STATO OWL with OWLAPI 

Here are examples of using [OWLAPI](http://owlcs.github.io/owlapi/) to process [STATO, the statistical methods ontology](https://www.ebi.ac.uk/ols/ontologies/stato). 

In [4]:
%%loadFromPOM
<!-- Use %%loadFromPOM instead of %maven. For some reason, the latter does not load guava correctly. -->
<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>owlapi-distribution</artifactId>
    <version>5.1.11</version>
</dependency>

In [3]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.vocab.DublinCoreVocabulary;
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;

In [5]:
import java.net.URI;
URI uri = new URI("http://purl.obolibrary.org/obo/stato.owl");
IRI iri = IRI.create(uri);

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [6]:
// load the ontology
OWLOntologyManager manager = OWLManager.createOWLOntologyManager();
OWLOntology onto = manager.loadOntology(iri);

In [8]:
// Retreive annotations
onto.annotations().forEach((ann) -> {
    IRI propIRI = ann.getProperty().getIRI();
    OWLLiteral value = (OWLLiteral) ann.getValue();
    if (propIRI.equals(DublinCoreVocabulary.TITLE.getIRI())) {
        System.out.println("Title: " + value.getLiteral());
    } else if (propIRI.equals(OWLRDFVocabulary.OWL_VERSION_INFO.getIRI())) {
        System.out.println("Version: " + value.getLiteral());
    }
});

Title: STATO: the statistical methods ontology
Version: RC1.4


## Traversing 

To traverse the ontology, first you need to include the reasoners. OWLAPI uses hermitIt reasoner. 

In [17]:
%%loadFromPOM
<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>org.semanticweb.hermit</artifactId>
    <version>1.4.3.517</version>
</dependency>

In [18]:
import org.semanticweb.owlapi.reasoner.NodeSet;
import org.semanticweb.owlapi.reasoner.OWLReasoner;
import org.semanticweb.owlapi.reasoner.OWLReasonerFactory;
import org.semanticweb.owlapi.search.EntitySearcher;
import org.semanticweb.HermiT.ReasonerFactory;

// Traverse the entities and print the hierarchy defined in the ontology
public static void printHierarchy(OWLReasoner reasoner, OWLClass owlClass, int level, Set<OWLClass> visited) {
    if (!visited.contains(owlClass) && reasoner.isSatisfiable(owlClass)) {
        visited.add(owlClass);
        for (int i = 0; i < level * 4; i++) {
            System.out.print(" ");
        }
        System.out.println(labelFor(owlClass, reasoner.getRootOntology()));

        NodeSet<OWLClass> classNodeSet = reasoner.getSubClasses(owlClass, true);
        for (OWLClass child: classNodeSet.getFlattened()) {
            printHierarchy(reasoner, child, level+1, visited);
        }
    }
}

// Extracts the label from the entity annotations
private static String labelFor(OWLClass clazz, OWLOntology o) {
    OWLAnnotationObjectVisitorEx<String> visitor = new OWLAnnotationObjectVisitorEx<String>() {
        String value;
        @Override
        public String visit(OWLAnnotation node) {
            if (node.getProperty().isLabel()) {
                return ((OWLLiteral) node.getValue()).getLiteral();
            }
            return null;
        }
    };
    return EntitySearcher.getAnnotations(clazz, o)  
            .map(anno -> anno.accept(visitor))
            .filter(value -> value != null)
            .findFirst()
            .orElse(clazz.getIRI().toString());
}

In [19]:
ReasonerFactory factory = new ReasonerFactory();
OWLReasoner r = factory.createReasoner(onto);
Set<OWLClass> visited = new HashSet<>();
onto.classesInSignature().forEach(en -> {
    printHierarchy(r, en, 0, visited);
});

entity
    continuant
        specifically dependent continuant
            realizable entity
                role
                    reagent role
                    participant under investigation role
                    measurand role
                        analyte role
                    material to be added role
                        cloning vector role
                    molecular label role
                        dye role
                    experimental unit role
                    cloning insert role
                    target of material addition role
                    specimen role
                        material sample role
                    evaluant role
                disposition
                    function
                        material separation function
                            sort function
                            filter function
                        heat function
                        contain function
                        environment

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

